In [27]:
import requests
import pandas as pd

api_key = 'YF0US57AOZXKGIGD'



In [28]:
class script_data:
    def __init__(self, symbol):
        self.symbol = symbol
        # self.interval = interval
        self.url = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={symbol}&apikey={api_key}'

    def fetch_intraday_data(self):
        response = requests.get(self.url)
        if response.status_code == 200:
            return response.json()
        else:
            raise Exception(
                f"Failed to fetch data. Response code: {response.status_code}")
    
    def convert_intraday_data(self, script):
        data = {}
        data["timestamp"] = []
        data["open"] = []
        data["high"] = []
        data["low"] = []
        data["close"] = []
        data["volume"] = []

        for key, value in script.items():
            if key.startswith("Time Series"):
                for timestamp, prices in value.items():
                    data["timestamp"].append(pd.Timestamp(timestamp))
                    data["open"].append(float(prices["1. open"]))
                    data["high"].append(float(prices["2. high"]))
                    data["low"].append(float(prices["3. low"]))
                    data["close"].append(float(prices["4. close"]))
                    data["volume"].append(int(prices["5. volume"]))

        df = pd.DataFrame(data)
        df.set_index("timestamp", inplace=True)

        return df
    
    def __getitem__(self, item):
        return self.df[item]
    
    def __setitem__(self, key, value):
        self.df[key] = value
    
    def __contains__(self, item):
        return item in self.df.columns
        


In [29]:
sd = script_data(symbol="GOOGL")
data = sd.fetch_intraday_data()

df = sd.convert_intraday_data(data)

# print(df)
sd.df = df

print("timestamp" in sd)  # Output: True
print("open" in sd)  # Output: True
print(sd["open"])  # Output: pandas.Series
sd["new_column"] = [0] * len(df)
print("new_column" in sd)  # Output: True


False
True
Series([], Name: open, dtype: float64)
True
